In [4]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

### Loading the instance

### Solver

In [5]:
from mclp import get_random_bin
from random import random 

def get_adjusted_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.adj_vol*boxes[box]
    return vol

def get_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.vol*boxes[box]
    return vol

def adjusted_swap(solution, n=2,  max_vol_accept=1.0, tolerance=0.1, verbose=True):

    bin_A = get_random_bin(s = solution)
    bin_B = get_random_bin(s = solution, b = bin_A)

    iniA = bin_A.adj_vol
    iniB = bin_B.adj_vol
    var_ini = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2

    nA = int(random.randint(1, n))
    boxes_A = bin_A.pop_random_boxes(nA)
    adjvolA = get_adjusted_vol(boxes_A)

    nB = int(random.randint(0, n))
    boxes_B = bin_B.pop_random_boxes(nB)
    adjvolB = get_adjusted_vol(boxes_B)

    r = tolerance*random.random()
    accepted = False

    if adjvolA > adjvolB and bin_B.adj_vol + adjvolA <= max_vol_accept + r:
        accepted = True
    elif adjvolB > adjvolA and  bin_A.adj_vol + adjvolB <= max_vol_accept + r:
        accepted = True

    if accepted:
        bin_A.insert_boxes(boxes_B)
        bin_B.insert_boxes(boxes_A)
    else: 
        return -10

    var_final = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
    var_diff = var_final - var_ini

    return var_diff

def get_sorted_vols(solution):
    volumens = []
    acum_volu = 0

    for s in solution:
        s_boxes = s.boxes
        vol_boxes = get_vol(s_boxes)
        acum_volu += get_vol(s_boxes)
        volumens.append(vol_boxes)

    volumens.sort()

    return volumens

""" 
    Obtiene los bins ordenados por sus volumenes de menor a mayor
    [1,2,4,11,1,23,5,123] ---(sort) --> [1, 1, 2, 4, 5, 11, 23, 123]
    tip: Agregar reverse igual True para ordenarlas de mayor a menor
"""
def get_sorted_bins(solution):

    list_solution = list(solution)
    list_solution.sort(key=lambda x: x.vol)

    return list_solution


In [6]:
from bsg import (load_BRinstance,load_BRKGAinstance, load_LargeInstance,
                 load_productInstance, load_instances_elhedhli)

from mclp import generate_candidate_solution
import numpy as np

import statistics as st
import time

total_ejecution = 1         # Total de ejecuciones por instancias
init_instance = 1           # Instancia desde donde comenzar
total_inst = 16              # Total de instancias para la clase
total_class = 2             # Total de clases
t_boxes = 5000              # Cantidad de cajas

r_param = 1.5                 # Parametro de llenado del contenedor
min_fr = 1               # minimo porcentaje ocupado para considerar la generación de un bloque
max_bl = 10000              # maxima cantidad de bloques a generar por bsg
bsg_time = 20               # Tiempo de BSG
extra_args = '--max_bl={} --min_fr={}'.format(max_bl, min_fr)

for i in range(1,total_class):

    for i_inst in range(init_instance, total_inst): 

        '''
        file_name_column = f"../benchs/Instance_CG/{t_boxes}/bin_pack_instance_i({i_inst})_c({i}).txt"
        L,W,H,boxes,id2box = load_instances_elhedhli(filename=file_name_column)
        '''

        file_name_datalarge = f'../benchs/Data_Large/L_{t_boxes}/L_{t_boxes}_{i_inst}.txt'
        L,W,H,boxes,id2box = load_LargeInstance(filename=file_name_datalarge)

        boxes=boxes.copy()

        solutions = []
        times = []
        less_volu = []
        _boxes=boxes.copy()

        vol_boxes = 0
        for key,value in id2box.items():
            vol_boxes += value.vol

        Vmax = L*W*H            # Volumen del contenedor

        for eje in range(total_ejecution):
            boxes =_boxes.copy()

            #Generate solution
            initial_time = time.time()
            init_solution = generate_candidate_solution(ssh=ssh, _L=L, _W=W, _H=H, 
                boxes=boxes, id2box=id2box, r_param=r_param, bsg_time=bsg_time, 
                extra_args=extra_args)
            last_time = time.time()
            
            #Execution time
            execution_time = last_time - initial_time
            
            # Sort the vol of the minor vol bin
            bins_to_plot = get_sorted_vols(init_solution)
            eje_x = [x for x in range(0,len(bins_to_plot))]
            minor_vol = bins_to_plot[0]

            solutions.append(len(eje_x))
            less_volu.append(minor_vol)
            times.append(execution_time)

        mean_sol = st.mean(solutions)
        mean_vol = st.mean(less_volu)
        mean_time = st.mean(times)

        usage_percentage = mean_vol/Vmax
        initial_lost = mean_sol*Vmax - vol_boxes
        lower_bound = vol_boxes/Vmax
        utilization_percentage = vol_boxes/(Vmax*mean_sol)

        result_str = f"{i_inst} {Vmax} {vol_boxes} {mean_sol} {initial_lost} {usage_percentage} {mean_time} {lower_bound} {utilization_percentage}"

        final_str = result_str.replace('.',',')
        # instancia promedio_solucion porcentage_de_uso tiempo_de_ejecucion_promedio
        print(final_str)

1 35960841 1215314935 41 259079546 0,02725948483796583 813,7562787532806 33,79550925964162 0,8242807136497956
2 35960841 1216230402 41 258164079 0,033734194369925884 813,129106760025 33,820966589741325 0,8249016241400323
3 35960841 1216411878 40 222021762 0,6937725955852924 794,6492817401886 33,82601307906008 0,845650326976502
4 35960841 1214375965 40 224057675 0,7059502585047998 795,8542456626892 33,769398357507825 0,8442349589376956
5 35960841 1208323293 40 230110347 0,535289233085511 793,8082880973816 33,601085497416484 0,840027137435412
6 35960841 1214915769 40 223517871 0,6506350894296382 795,1618175506592 33,78440924115206 0,8446102310288016
7 35960841 1218310125 41 256084356 0,07709397007706244 814,02290391922 33,8787995809108 0,8263121849002635
8 35960841 1216545193 40 221888447 0,603810517112211 794,2932226657867 33,82972030604067 0,8457430076510168
9 35960841 1216393922 40 222039718 0,5851147363322231 792,1867711544037 33,82551375814598 0,8456378439536495
10 35960841 12148095